In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import chi2
import numpy as np
#pd.set_option("display.max_colwidth", -1)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [2]:
# read the csv file with keywords
keyword = pd.read_csv(r'C:\Users\nitis\Sentence classification\Data\keyword_vocab.csv')

In [3]:
cols = ['sentence', 'section_nr','has_citation','last_section_title','Labels']
cols1 = ['sentence', 'section_nr','has_citation','last_section_title']
cols2 = ['sentence', 'Labels_rule']

In [4]:
#get non-rule based test preprocessed dataset
test = pd.read_csv(r'C:\Users\nitis\Sentence classification\theis_final\test_concat.csv')
data = test[cols]
data

,sentence,section_nr,has_citation,last_section_title,Labels
0,The aim of this thesis was to gain an understanding of the diametrical deformation behaviour of acetabular cups and shells following impaction into the reamed acetabulum. 1 0 Research Aim,1,0,Research Aim,Aim
1,"The aim is to develop finite element models using explicit dynamics to mimic mallet blows during cup/shell insertion, initially using simplified experimentally validated foam models to represent the acetabulum. 1 0 Research Objective",1,0,Research Objective,Aim
2,Determine the relationship between the size of interference between the cup and cavity and deformation for different cup types. 1 0 Research Objective,1,0,Research Objective,Aim
3,Investigate the influence of non-uniform cup support and varying the orientation of the component in the cavity on deformation. 1 0 Research Objective,1,0,Research Objective,Aim
4,Examine the influence of errors during reaming of the acetabulum which introduce ovality to the cavity. 1 0 Research Objective,1,0,Research Objective,Aim
...,...,...,...,...,...
577,the moment of enjoyment on the pattern to raise be material than on the intention to keep on the premium subscription. 39 0 hypothesis outcome,39,0,hypothesis outcome,Hypothesis Result
578,Information technology substantiate the hypothesis that a good nighttime sleep can increase the productiveness 10 0 hypothesis outcome,10,0,hypothesis outcome,Hypothesis Result
579,information engineering science rejects the surmisal that corrode rugularly serve non urinate any wallop in weight 10 0 hypothesis outcome,10,0,hypothesis outcome,Hypothesis Result
580,the supposition stern follow agreed on the cornerstone of the result 10 0 hypothesis outcome,10,0,hypothesis outcome,Hypothesis Result


In [5]:
# create list of patterns of keywords according to each sentence type

aim_key =  keyword.loc[keyword['Keyword'].str.contains("Aim/Objective")]
aim_key =  list(aim_key["values"])
pattern_aim = '|'.join([f'(?i){key}' for key in aim_key])

contribution_key =  keyword.loc[keyword['Keyword'].str.contains("Contribution")]
contribution_key =  list(contribution_key["values"])
pattern_contribution = '|'.join([f'(?i){key}' for key in contribution_key])

Defination_key =  keyword.loc[keyword['Keyword'].str.contains("Defination")]
Defination_key =  list(Defination_key["values"])
pattern_defination = '|'.join([f'(?i){key}' for key in Defination_key])

Emperical_key =  keyword.loc[keyword['Keyword'].str.contains("Emperical Result")]
Emperical_key =  list(Emperical_key["values"])
pattern_emperical = '|'.join([f'(?i){key}' for key in Emperical_key])

Future_work =  keyword.loc[keyword['Keyword'].str.contains("Future work")]
Future_work =  list(Future_work["values"])
pattern_future = '|'.join([f'(?i){key}' for key in Future_work])


hypothesis_result =  keyword.loc[keyword['Keyword'] == "hypothesis_result"]
hypothesis_result =  list(hypothesis_result["values"])
pattern_hypothesis_result = '|'.join([f'(?i){key}' for key in hypothesis_result])

hypothesis =  keyword.loc[keyword['Keyword'] == "hypothesis"]
hypothesis =  list(hypothesis["values"])
pattern_hypothesis = '|'.join([f'(?i){key}' for key in hypothesis])

limitation =  keyword.loc[keyword['Keyword'] == "Limitation"]
limitation =  list(limitation["values"])
pattern_limitation = '|'.join([f'(?i){key}' for key in limitation])


motivation =  keyword.loc[keyword['Keyword'] == "Motivation"]
motivation =  list(motivation["values"])
pattern_motivation = '|'.join([f'(?i){key}' for key in motivation])

relation2literature =  keyword.loc[keyword['Keyword'] == "Relation to Literature"]
relation2literature =  list(relation2literature["values"])
pattern_relation2literature= '|'.join([f'(?i){key}' for key in relation2literature])

research_question =  keyword.loc[keyword['Keyword'] == "research question"]
research_question =  list(research_question["values"])
pattern_research_question= '|'.join([f'(?i){key}' for key in research_question])

method =  keyword.loc[keyword['Keyword'] == "method"]
method =  list(method["values"])
pattern_method= '|'.join([f'(?i){key}' for key in method])

In [6]:
# RULE CLASS 2

#defination
conditions7 = [(data['sentence'].str.contains(pattern_defination))]

#limitation
conditions12 = [(data['last_section_title'].str.contains("(?i)limitation|conclusion"))  
                & (data['sentence'].str.contains(pattern_limitation))]

#future work
conditions13 = [(data['last_section_title'].str.contains("(?i)conclusion|contribution"))  
                & (data['sentence'].str.contains(pattern_future))
               ]

#contribution
conditions11 = [(data['last_section_title'].str.contains("(?i)conclusion|contribution|discussion"))  
                & (data['sentence'].str.contains(pattern_contribution)) 
               ]


#hypothesis result
conditions9 = [(data['sentence'].str.contains(pattern_hypothesis))
               & (data['sentence'].str.contains(pattern_hypothesis_result))]

#hypothesis
conditions8 = [(data['sentence'].str.contains(pattern_hypothesis))
              ]


#method
conditions6 = [(data['last_section_title'].str.contains("(?i)method|methodology|methodological")) 
               & (data['section_nr'] > 1) 
               & (data['sentence'].str.contains(pattern_method))
               ]

#Research Question
conditions4 = [(data['sentence'].str.contains(pattern_research_question))]


#Motivation
conditions3 = [(data['section_nr'] == 1) 
               &  (data['sentence'].str.contains(pattern_motivation))  
               ]


#AIM
conditions2 =  [(data['section_nr'] == 1) 
               & (data['sentence'].str.contains(pattern_aim)) 
               ]




#Emperical result
conditions10 = [(data['last_section_title'].str.contains("(?i)result|analysis|emperical|finding")) 
                & (data['section_nr'] > 1) 
                & (data['sentence'].str.contains(pattern_emperical)) 
                ]


#Relation Literature 
conditions5 = [(data['has_citation'] == 1) 
               & (data['sentence'].str.contains(pattern_relation2literature)) 
               ]





choices = [1]

data['aim'] = np.select(conditions2, choices, default=0)
data['motivation'] = np.select(conditions3, choices, default=0)
data['research_question'] = np.select(conditions4, choices, default=0)
data['relation_literature'] = np.select(conditions5, choices, default=0)
data['method'] = np.select(conditions6, choices, default=0)

data['Defination'] = np.select(conditions7, choices, default=0)
data['hypothesis'] = np.select(conditions8, choices, default=0)
data['hypothesis result'] = np.select(conditions9, choices, default=0)
data['Emperical result'] = np.select(conditions10, choices, default=0)
data['contribution'] = np.select(conditions11, choices, default=0)
data['limitation'] = np.select(conditions12, choices, default=0)
data['future work'] = np.select(conditions13, choices, default=0)


data.iloc[:, 10:] = data[data.columns[10:]].replace(0, np.nan)
data.head(1)

,sentence,section_nr,has_citation,last_section_title,Labels,aim,motivation,research_question,relation_literature,method,Defination,hypothesis,hypothesis result,Emperical result,contribution,limitation,future work
0,The aim of this thesis was to gain an understanding of the diametrical deformation behaviour of acetabular cups and shells following impaction into the reamed acetabulum. 1 0 Research Aim,1,0,Research Aim,Aim,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#RULE CLASS 2 :  

conditions_second = [
    (data['hypothesis result'] == 1),
    (data['research_question'] == 1),
    (data['limitation'] == 1),
    (data['motivation'] == 1),
    (data['future work'] == 1),
    (data['contribution'] == 1),
    (data['hypothesis'] == 1),
    (data['method'] == 1),
    
    (data['aim'] == 1),
    (data['Emperical result'] == 1),
    
    
    (data['relation_literature'] == 1),
    (data['Defination'] == 1),
    (data['Defination'] == 0)
    
]

choices3 = ['Hypothesis Result','Research Question','limitation','Motivation','Future work','Contribution','Hypothesis','Method','Aim','Emperical Result','Relation to literature','Defination' , np.nan]
data['Labels_rule'] = np.select(conditions_second, choices3, default=np.nan)
data

,sentence,section_nr,has_citation,last_section_title,Labels,aim,motivation,research_question,relation_literature,method,Defination,hypothesis,hypothesis result,Emperical result,contribution,limitation,future work,Labels_rule
0,The aim of this thesis was to gain an understanding of the diametrical deformation behaviour of acetabular cups and shells following impaction into the reamed acetabulum. 1 0 Research Aim,1,0,Research Aim,Aim,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aim
1,"The aim is to develop finite element models using explicit dynamics to mimic mallet blows during cup/shell insertion, initially using simplified experimentally validated foam models to represent the acetabulum. 1 0 Research Objective",1,0,Research Objective,Aim,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Motivation
2,Determine the relationship between the size of interference between the cup and cavity and deformation for different cup types. 1 0 Research Objective,1,0,Research Objective,Aim,1,1,0,0,0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Motivation
3,Investigate the influence of non-uniform cup support and varying the orientation of the component in the cavity on deformation. 1 0 Research Objective,1,0,Research Objective,Aim,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Motivation
4,Examine the influence of errors during reaming of the acetabulum which introduce ovality to the cavity. 1 0 Research Objective,1,0,Research Objective,Aim,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Motivation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,the moment of enjoyment on the pattern to raise be material than on the intention to keep on the premium subscription. 39 0 hypothesis outcome,39,0,hypothesis outcome,Hypothesis Result,0,0,0,0,0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,Hypothesis
578,Information technology substantiate the hypothesis that a good nighttime sleep can increase the productiveness 10 0 hypothesis outcome,10,0,hypothesis outcome,Hypothesis Result,0,0,0,0,0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,Hypothesis
579,information engineering science rejects the surmisal that corrode rugularly serve non urinate any wallop in weight 10 0 hypothesis outcome,10,0,hypothesis outcome,Hypothesis Result,0,0,0,0,0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,Hypothesis Result
580,the supposition stern follow agreed on the cornerstone of the result 10 0 hypothesis outcome,10,0,hypothesis outcome,Hypothesis Result,0,0,0,0,0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,Hypothesis


In [8]:
test = data.reset_index(drop=True)
test = test[["sentence","section_nr","last_section_title","Labels","has_citation","Labels_rule"]]
test['has_citation'] = test['has_citation'].replace(np.nan, 0)
test

,sentence,section_nr,last_section_title,Labels,has_citation,Labels_rule
0,The aim of this thesis was to gain an understanding of the diametrical deformation behaviour of acetabular cups and shells following impaction into the reamed acetabulum. 1 0 Research Aim,1,Research Aim,Aim,0,Aim
1,"The aim is to develop finite element models using explicit dynamics to mimic mallet blows during cup/shell insertion, initially using simplified experimentally validated foam models to represent the acetabulum. 1 0 Research Objective",1,Research Objective,Aim,0,Motivation
2,Determine the relationship between the size of interference between the cup and cavity and deformation for different cup types. 1 0 Research Objective,1,Research Objective,Aim,0,Motivation
3,Investigate the influence of non-uniform cup support and varying the orientation of the component in the cavity on deformation. 1 0 Research Objective,1,Research Objective,Aim,0,Motivation
4,Examine the influence of errors during reaming of the acetabulum which introduce ovality to the cavity. 1 0 Research Objective,1,Research Objective,Aim,0,Motivation
...,...,...,...,...,...,...
577,the moment of enjoyment on the pattern to raise be material than on the intention to keep on the premium subscription. 39 0 hypothesis outcome,39,hypothesis outcome,Hypothesis Result,0,Hypothesis
578,Information technology substantiate the hypothesis that a good nighttime sleep can increase the productiveness 10 0 hypothesis outcome,10,hypothesis outcome,Hypothesis Result,0,Hypothesis
579,information engineering science rejects the surmisal that corrode rugularly serve non urinate any wallop in weight 10 0 hypothesis outcome,10,hypothesis outcome,Hypothesis Result,0,Hypothesis Result
580,the supposition stern follow agreed on the cornerstone of the result 10 0 hypothesis outcome,10,hypothesis outcome,Hypothesis Result,0,Hypothesis


In [9]:
test['sentence'] = test[cols2].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
test

,sentence,section_nr,last_section_title,Labels,has_citation,Labels_rule
0,The aim of this thesis was to gain an understanding of the diametrical deformation behaviour of acetabular cups and shells following impaction into the reamed acetabulum. 1 0 Research Aim Aim,1,Research Aim,Aim,0,Aim
1,"The aim is to develop finite element models using explicit dynamics to mimic mallet blows during cup/shell insertion, initially using simplified experimentally validated foam models to represent the acetabulum. 1 0 Research Objective Motivation",1,Research Objective,Aim,0,Motivation
2,Determine the relationship between the size of interference between the cup and cavity and deformation for different cup types. 1 0 Research Objective Motivation,1,Research Objective,Aim,0,Motivation
3,Investigate the influence of non-uniform cup support and varying the orientation of the component in the cavity on deformation. 1 0 Research Objective Motivation,1,Research Objective,Aim,0,Motivation
4,Examine the influence of errors during reaming of the acetabulum which introduce ovality to the cavity. 1 0 Research Objective Motivation,1,Research Objective,Aim,0,Motivation
...,...,...,...,...,...,...
577,the moment of enjoyment on the pattern to raise be material than on the intention to keep on the premium subscription. 39 0 hypothesis outcome Hypothesis,39,hypothesis outcome,Hypothesis Result,0,Hypothesis
578,Information technology substantiate the hypothesis that a good nighttime sleep can increase the productiveness 10 0 hypothesis outcome Hypothesis,10,hypothesis outcome,Hypothesis Result,0,Hypothesis
579,information engineering science rejects the surmisal that corrode rugularly serve non urinate any wallop in weight 10 0 hypothesis outcome Hypothesis Result,10,hypothesis outcome,Hypothesis Result,0,Hypothesis Result
580,the supposition stern follow agreed on the cornerstone of the result 10 0 hypothesis outcome Hypothesis,10,hypothesis outcome,Hypothesis Result,0,Hypothesis


In [11]:
import string
string.punctuation
#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

#defining function for tokenization
import re
def tokenization(text):
    tokens = re.split('W+',text)
    return tokens


#importing nlp library
import nltk
#Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
#defining the function to remove stopwords from tokenized text
def remove_stopwords(text):
    output="".join([i for i in text if i not in stopwords])
    return output

from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

#defining the function for lemmatization
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

In [12]:
#text preprocessing

test['sentence']= test['sentence'].apply(lambda x:remove_punctuation(x))
#applying function to the column
test['sentence']= test['sentence'].apply(lambda x: tokenization(x))
#applying the function
test['sentence']= test['sentence'].apply(lambda x:remove_stopwords(x))
#applying function to the column
test['sentence']= test['sentence'].apply(lambda x: tokenization(x))
test['sentence']=test['sentence'].apply(lambda x:lemmatizer(x))
#list to string
test['sentence'] = [' '.join(map(str, l)) for l in test['sentence']]
test

,sentence,section_nr,last_section_title,Labels,has_citation,Labels_rule
0,The aim of this thesis was to gain an understanding of the diametrical deformation behaviour of acetabular cups and shells following impaction into the reamed acetabulum 1 0 Research Aim Aim,1,Research Aim,Aim,0,Aim
1,The aim is to develop finite element models using explicit dynamics to mimic mallet blows during cupshell insertion initially using simplified experimentally validated foam models to represent the acetabulum 1 0 Research Objective Motivation,1,Research Objective,Aim,0,Motivation
2,Determine the relationship between the size of interference between the cup and cavity and deformation for different cup types 1 0 Research Objective Motivation,1,Research Objective,Aim,0,Motivation
3,Investigate the influence of nonuniform cup support and varying the orientation of the component in the cavity on deformation 1 0 Research Objective Motivation,1,Research Objective,Aim,0,Motivation
4,Examine the influence of errors during reaming of the acetabulum which introduce ovality to the cavity 1 0 Research Objective Motivation,1,Research Objective,Aim,0,Motivation
...,...,...,...,...,...,...
577,the moment of enjoyment on the pattern to raise be material than on the intention to keep on the premium subscription 39 0 hypothesis outcome Hypothesis,39,hypothesis outcome,Hypothesis Result,0,Hypothesis
578,Information technology substantiate the hypothesis that a good nighttime sleep can increase the productiveness 10 0 hypothesis outcome Hypothesis,10,hypothesis outcome,Hypothesis Result,0,Hypothesis
579,information engineering science rejects the surmisal that corrode rugularly serve non urinate any wallop in weight 10 0 hypothesis outcome Hypothesis Result,10,hypothesis outcome,Hypothesis Result,0,Hypothesis Result
580,the supposition stern follow agreed on the cornerstone of the result 10 0 hypothesis outcome Hypothesis,10,hypothesis outcome,Hypothesis Result,0,Hypothesis


In [13]:
#test.to_csv(r'C:\Users\nitis\Sentence classification\theis_final\rule_test_data.csv')